<a href="https://colab.research.google.com/github/mbozdal/SingleNeuronIDS/blob/main/colab/single_neuron_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mat73
!pip install --upgrade gspread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2


In [ ]:
from google.colab import drive

drive.mount('/content/drive')



In [ ]:
!lscpu |grep 'Model name'

In [ ]:
!nvidia-smi

In [ ]:
!ls "/content/drive/My Drive/Ali AI IDS/matlab/ftrain"

In [ ]:
from sklearn.metrics import confusion_matrix
from gspread import worksheet
from google.colab import auth
import gspread
from google.auth import default

#obtain results for the attack data then write results to the excel file.
#model=model type, index=index of CAN id, attackType=page index just writing to the excel file.
def model_results(model,index,attackType):
 
    x =  tf.round(model.predict(X_test))

    acc = tf.keras.metrics.Accuracy()
    acc.update_state(y_test, x)
  
    pre = tf.keras.metrics.Precision()
    pre.update_state(y_test, x)

    recall = tf.keras.metrics.Recall()
    recall.update_state(y_test, x)
 

    fp = tf.keras.metrics.FalsePositives()
    fn = tf.keras.metrics.FalseNegatives()
    tp = tf.keras.metrics.TruePositives()
    tn = tf.keras.metrics.TrueNegatives()
    fp.update_state(y_test, x)
    fn.update_state(y_test, x)
    tp.update_state(y_test, x)
    tn.update_state(y_test, x)

  #  print(f"{data_name}\t{fp.result().numpy()}\t{fn.result().numpy()}\t{tp.result().numpy()}\t{tn.result().numpy()}\n") 
   
    auth.authenticate_user()

    creds, _ = default()

    gc = gspread.authorize(creds)
    wb = gc.open_by_key('1p6LQZ3DkJgdm0_TkvLaM6Y32cOvF6cz39vRycKVNgRk')
    
    dataToWrite = [[data_name,str(fp.result().numpy()),str(fn.result().numpy()),str(tp.result().numpy()),str(tn.result().numpy())]]
    

    if (attackType==0):
          ws = wb.worksheet('FuzzyAttack')
    elif (attackType==1):
          ws = wb.worksheet('DoS')
    elif (attackType==2):
          ws = wb.worksheet('Gear')
    elif (attackType==3):
          ws = wb.worksheet('RPM')
  
    rowNumber= "A"+str(index+1)
    ws.update(rowNumber,dataToWrite)
    index = index + 1

In [ ]:
import mat73
import scipy.io
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.utils import validation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential


for ii in range(0,26):   
  canID = ['0002','00a0','00a1','0130','0131','0140','0153','018f','01f1','0260','02a0','02c0','0316','0329','0350','0370','0430','043f','0440','04b1','04f0','0545','05a0','05a2','05f0','0690']
  train_batch_size = 64
  test_batch_size = 64
  epoch_num= 5
  data_name = canID[ii]

# Load train dataset (merged data)
  data = mat73.loadmat('/content/drive/My Drive/Ali AI IDS/matlab/ftrain/merge_'+ canID[ii] +'_ftrain_1x8.mat')
  

  label = data['labels']
  dataIn = data['inputMatrix']

  X_train, X_test, y_train, y_test = train_test_split(dataIn, label, test_size=0.3, random_state=42, shuffle=False)


 #ANN model structure
  tf.random.set_seed(42)
  model_single_neuron = Sequential([Dense(1, activation="sigmoid")])

  model_single_neuron.compile(loss="binary_crossentropy", optimizer=Adam(), metrics=["accuracy"])
  history = model_single_neuron.fit(X_train, y_train, epochs=epoch_num, batch_size=train_batch_size)


#saved weights
  name='Weights_'+data_name + '_' + str(epoch_num) + '_' + str(train_batch_size) + '_' +str(test_batch_size)
  model_single_neuron.save('/content/drive/My Drive/Ali AI IDS/matlab/ftrain/weights/' + name)

  dataAttackNames = ['fuzzyAttack','dosAttack','gearSpoofing','rpmSpoofing']

  for attackType in range(0,4):
    dataAttackName = dataAttackNames[attackType]

    dataTest = mat73.loadmat('/content/drive/My Drive/Ali AI IDS/matlab/ftrain/'+dataAttackName+'_'+canID[ii] +'_ftrain_1x8.mat')
    dataTestIn = dataTest['inputMatrix'] 
    dataTestLabel = dataTest['labels'] 
    X_dummy_train, X_test, y_dummy_train, y_test = train_test_split(dataTestIn, dataTestLabel, test_size=0.3, random_state=42, shuffle=False)

    model_single_neuron.evaluate(X_test, y_test, batch_size=test_batch_size)
    model_results(model_single_neuron,ii,attackType)

